# Inference pipeline

Created by: Oriana Chegwidden


In [ ]:
%load_ext autoreload
%autoreload 2

from pyproj import CRS
import boto3
from rasterio.session import AWSSession
from s3fs import S3FileSystem
aws_session = AWSSession(boto3.Session(),#profile_name='default'), 
                         requester_pays=True)
fs = S3FileSystem(requester_pays=True) #profile='default', 
import xgboost as xgb

from osgeo.gdal import VSICurlClearCache
import rasterio as rio
import numpy as np
import xarray as xr
import dask
import os
import fsspec
import geopandas as gpd
import rioxarray # for the extension to load
import matplotlib.pyplot as plt
import utm
import pandas as pd
from datetime import datetime
import json
import zarr
import awswrangler as wr
from dask_gateway import Gateway
from carbonplan_trace.v1.landsat_preprocess import access_credentials, test_credentials
from carbonplan_trace.v1.inference import predict, predict_delayed
from carbonplan_trace.v1 import utils


In [ ]:
from carbonplan_trace import version

print(version)

In [ ]:
# kind_of_cluster = "local"
kind_of_cluster = "remote"
if kind_of_cluster == "local":
    # spin up local cluster. must be on big enough machine
    from dask.distributed import Client

    client = Client(n_workers=4, threads_per_worker=1)  # _per_worker=4
    client
else:
    gateway = Gateway()
    options = gateway.cluster_options()
    options.environment = {
        "AWS_REQUEST_PAYER": "requester",
        "AWS_REGION_NAME": "us-west-2",
        
    }
    options.worker_cores = 1
    options.worker_memory = 80
    options.image = "carbonplan/trace-python-notebook:latest"
    cluster = gateway.new_cluster(cluster_options=options)
    #     cluster.adapt(minimum=2, maximum=100)
    cluster.scale(100)

# lower latitudes = lower memory
# I've tried 200gb mem for worker with 100 workers => 150 nodes, where our limit = 500 nodes
# 32gb => i've tried 400 workers, can probably go higher
# increase memory when you start seeing a lot of Killed Worker Errors

# recipe
# try running 60gb with 300 workers for bounding_boxes 0-100
# 120gb with 200 workers for bounding boxes 100-200
# 200gb with 100 workers for bounding boxes 200-end (there are 280 boxes)

In [ ]:
cluster

In [ ]:
cluster

In [ ]:
client

# check this link first
# possible scenario:
# 1) everything is succeeding and cluster still running, no need to do anything
# 2) most things are failing but cluster still running, restart, increase mem and decrease num worker, re start and run all
# 3) 404 error -> cluster died -> restart and run all

In [ ]:
def shutdown_cluster(kind_of_cluster):
    if kind_of_cluster == "local":
        client.shutdown()
    elif kind_of_cluster == "remote":
        cluster.shutdown()

In [ ]:
# shutdown_cluster(kind_of_cluster)

In [ ]:
access_key_id, secret_access_key = access_credentials()

In [ ]:
test_credentials(aws_session)

Then we take the list of files for a given year to average across growing season
for each of the tiles and write it out to a mapper with those specifications.


In [ ]:
dask.config.set({"array.slicing.split_large_chunks": True})
# tcp is a transmission control protocol
dask.config.set({"distributed.comm.timeouts.tcp": "50s"})
dask.config.set({"distributed.comm.timeouts.connect": "50s"})

In [ ]:
gdf = gpd.read_file(
    "https://prd-wret.s3-us-west-2.amazonaws.com/assets/"
    "palladium/production/s3fs-public/atoms/files/"
    "WRS2_descending_0.zip"
)
bucket = "s3://carbonplan-climatetrace/v1"

biomass_folder = bucket + "/biomass/"
biomass_files = fs.ls(biomass_folder)
lat_lon_tags = [
    utils.get_lat_lon_tags_from_tile_path(fp) for fp in biomass_files
]
bounding_boxes = [
    utils.parse_bounding_box_from_lat_lon_tags(lat, lon)
    for lat, lon in lat_lon_tags
]

In [ ]:
from carbonplan_trace.v1.glas_allometric_eq import REALM_GROUPINGS

processed_scenes = []
for year in np.arange(2015, 2021):
    processed_scenes.extend(
        fs.ls(f"{bucket}/inference/rf/{year}", recursive=True)
    )

processed_scenes = [scene[-19:-8] for scene in processed_scenes]

In [ ]:
len(processed_scenes)

We'll loop through every scene and every year and calculate biomass for that
scene. Will produce table of values [x, y, (both specific to utm projection),
lat, lon, biomass].


In [ ]:
landsat_bucket = (
    "s3://usgs-landsat/collection02/level-2/standard/etm/{}/{:03d}/{:03d}/"
)

with rio.Env(aws_session):
    tasks = []
    task_id = []
    # for all bounding boxes ==> 70k tasks, will take 15 mins to append all tasks
    for bounding_box in bounding_boxes[10:20]:
        print(bounding_box)
        min_lat, max_lat, min_lon, max_lon = bounding_box
        scenes_in_tile = gdf.cx[min_lon:max_lon, min_lat:max_lat][
            ["PATH", "ROW"]
        ].values
        for year in np.arange(2015, 2021):
            for [path, row] in scenes_in_tile:
                scene_stores = fs.ls(landsat_bucket.format(year, path, row))
                output_name = f"{year}/{path:03d}{row:03d}"
                if len(scene_stores) == 0:
                    continue
                elif output_name in processed_scenes:
                    continue
                else:
                    tasks.append(
                        # predict(
                        #                         client.compute(   # what cindy usually runs
                        predict_delayed(
                            model_folder=f"{bucket}/models/",
                            path=path,
                            row=row,
                            year=year,
                            access_key_id=access_key_id,
                            secret_access_key=secret_access_key,
                            output_write_bucket=f"{bucket}/inference",
                        )
                    )
                    task_id.append([path, row, year])

In [ ]:
len(tasks)

In [ ]:
# path = 165
# row = 65
# year = 2016

# predict(
#     model_folder=f"{bucket}/models/",
#     path=path,
#     row=row,
#     year=year,
#     access_key_id=access_key_id,
#     secret_access_key=secret_access_key,
#     output_write_bucket=f"{bucket}/inference"
# )

In [ ]:
import random

test_run = random.choices(tasks, k=20)

In [ ]:
results = dask.compute(test_run, retries=1)[0]
results

In [ ]:
exclude_list = []

# three types of error
# KilledWorker == memory usage
# Assertion Error => need to figure out what we want to do for inference, can throw away for now
# .zmetadata
# AccessDenied should have been fixed

for i, task in enumerate(tasks):
    if task.status == "error" and i not in []:
        print(i)
        #         print(task.result())
        try:
            print(task.result())
        except Exception as e:
            print(e)
#             if isinstance(e, PermissionError):  # you can change this to other error
#                 exclude_list.append(task_id[i])
#                 print(i)

In [ ]:
# for task in tasks:
#     task.cancel()